In [ ]:
%cd /content/drive/MyDrive/Voice_Recognition

Reference: https://github.com/jiaaro/pydub

In [ ]:
# install requirements

!pip install -r requirements.txt

In [4]:
# imports

import pydub

In [6]:
voice1 = pydub.AudioSegment.from_file("raw_data/Nahid.ogg")

In [ ]:
voice1

In [ ]:
voice1.export("Nahid.mp3")